In [1]:
#import relevant libraries
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
#set data_location containing profiles.csv file
data_location = "data/profiles.csv"

In [3]:
#import profiles.csv file as a dataframe
data = pd.read_csv(data_location)
print(len(data))

59946
